# Ejercicio autoencoders

![Autoencoder](https://media.licdn.com/dms/image/C4E12AQGcq6YavpwTTg/article-cover_image-shrink_600_2000/0/1633637593999?e=2147483647&v=beta&t=YrRxXMUhoU2v1yODNO-I_H0Dv7X-uW-ADoAUuDrXoJY)

In [ ]:
COLAB = True # crear maquina virtual en google colab

In [ ]:
import pandas as pd

# Modelos
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler # clase "standardScaler" para normalizar los datos
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Visualizacion
from matplotlib.ticker import MaxNLocator # para que los ejes de las graficas sean enteros
import matplotlib.pyplot as plt
import numpy as np


## Cargar y preparar los datos

1. Cargar los datos
2. Hacer un split train/test
3. Estandirzar los datos del train
4. Estandarizar los datos del test usando el estandizador train

In [ ]:
# Cargar datos

#help(load_wine)
data = load_wine()
data

Este dataset tiene 3 clases: ['class_0', 'class_1', 'class_2'].

In [ ]:
# Crear dataframe (antes era un diccionario)
df = pd.DataFrame(data['data'], 
                  columns=data['feature_names'])
df

In [ ]:
# Ver resumen estadistico de los datos
df.describe().T[['min', 'max', 'mean', 'std']]

La variable proline tiene una varianza muy superior al resto de variables. Para evitar que el autoencoder se centre en esta variable, vamos a normalizar el dataset.

In [ ]:
# Hacer un split de los datos en train y va l
df_train, df_val = train_test_split(df, 
                                    test_size=0.3, # 30% de los datos para validacion
                                    random_state=1) # semilla para que sea reproducible
df_train.head(3)

In [ ]:
# Normalizar los datos
scaler = StandardScaler() # scaler es una instancia de la clase "StandardScaler"

scaler.fit_(df_train) #  calcular estadísticas estandarización; Hacer solo un fit en los datos de entrenamiento (porque en teoria no deberiamos tener acceso a los datos de test/validacion)
df_train2 = scaler.transform(df_train)
df_val2 = scaler.transform(df_val)
df_train2

In [ ]:
# Crear nuevo dataframe con los datos normalizados (antes era un array por la transformacion)
df2_train = pd.DataFrame(df_train2, 
                   columns=data['feature_names'])
df2_val = pd.DataFrame(df_val2, 
                   columns=data['feature_names'])
df2_train

Ahora si tenemos valores negativos, signifaca que ese valor es menor que la media de su variable. Después de la normalización, la media de todas las variables es 0.

## Definir y entrenar el autoencoder

In [ ]:
# Definir mi AUTOENCODER como una lista de capas:
num_features = df2_train.shape[1] # numero de columnas
model = keras.Sequential(
    [
     keras.Input(num_features),
     keras.layers.Dense(2, activation="relu"), # Aqui es mi Bottleneck (compressed data) con una función no lineal
     keras.layers.Dense(num_features)
    ]
)

In [ ]:
model.summary()

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Definir la visualizacion de la historia del entrenamiento
def plot_history(historia):
    f = plt.figure(figsize=(4,4))
    h = historia.history
    aux = range(1,len(h["loss"])+1)
    mejor_epoca = np.argmin(h["val_loss"])
    plt.plot(aux, h["loss"], label="entrenamiento")
    plt.plot(aux, h["val_loss"], label="validación")
    plt.plot(mejor_epoca+1, h["val_loss"][mejor_epoca], 'or')
    plt.title('Loss', fontsize=18)
    plt.xlabel('Época', fontsize=18)
    plt.xticks(fontsize=12); plt.yticks(fontsize=12)
    plt.legend()
    f.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
# Definir callbacks 
    # 1) para terminar el entrenamiento cuando ya no baja el error de validacion (EarlyStopping)
    # 2) para guardar el mejor modelo (ModelCheckpoint)
lista_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5, # numero de epocas sin mejora del modelo
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]

In [ ]:
# Entrenar el modelo
historia = model.fit(x=df2_train,       # datos de entrenamiento
                     y=df2_train,       # etiquetas de entrenamiento
                     batch_size=32,     # tamaño del batch
                     epochs=50,         # numero de epocas
                     callbacks=lista_callbacks,             # callbacks
                     validation_data=(df2_val, df2_val))    # datos de validacion

In [ ]:
# Plotear la historia
plot_history(historia)

Según el resulatdo del ejercicio anterior, puedes cambiar los parametros del autoencoder, por ejemplo: las epocas

In [ ]:
# Cargar el mejor modelo
model = keras.models.load_model("best_model.keras")

## Sacar solo el encoder (salida de la primera capa)

In [ ]:
# Acceder a la primera capa oculta (Bottleneck)

encoder = keras.Model(inputs=model.input, 
                      outputs=model.layers[0].output)

In [ ]:
encoder.summary()

In [ ]:
df2_train_compressed = encoder.predict(df2_train) # comprimir los datos de entrenamiento
df2_val_compressed = encoder.predict(df2_val) # comprimir los datos de entrenamiento

df2_train_compressed.shape

Nos sale un array de 124 filas y 2 columnas: 124 filas porque tenemos 124 muestras y 2 columnas porque hemos definido 2 neuronas en la capa oculta.

In [ ]:
plt.plot(df2_train_compressed[:,0], df2_train_compressed[:,1], 'o')

Se puede ver por lo menos 2 grupos de vinos en el gráfico. En plan, deberían ser 3 grupos, pero se ve que hay 2 grupos que se solapan.
No hemos incluido las clases/etiquetas en el entrenamiento, entonces el resultado no es perfecto.

## Inclyuir las etiquetas en el entrenamiento

In [ ]:
# Guardar las etiquetas 
y = data['target']

In [ ]:
# Hacer un split de los datos en train y val
df_train, df_val, y_train, y_val = train_test_split(df, y, test_size=0.3, random_state=1)

Y ahora seguimos como en el ejercicio anterior...

- Estandarizar los datos del train
- Estandarizar los datos del test/val usando el estandizador train
- Definir y entrenar el autoencoder
- Sacar solo el encoder (salida de la primera capa)
- Comprimir los datos de train usando el encoder

In [ ]:
# Normalizar los datos
scaler = StandardScaler() # scaler es una instancia de la clase "StandardScaler"

scaler.fit_(df_train) #  calcular estadísticas estandarización; Hacer solo un fit en los datos de entrenamiento (porque en teoria no deberiamos tener acceso a los datos de test/validacion)
df_train2 = scaler.transform(df_train)
df_val2 = scaler.transform(df_val)
df_train2

In [ ]:
# Crear nuevo dataframe con los datos normalizados (antes era un array por la transformacion)
df2_train = pd.DataFrame(df_train2, 
                   columns=data['feature_names'])
df2_val = pd.DataFrame(df_val2, 
                   columns=data['feature_names'])
df2_train

In [ ]:
# Definir mi AUTOENCODER como una lista de capas:

model = keras.Sequential(
    [
     keras.Input(13),
     keras.layers.Dense(2, activation="relu"), # Aqui es mi Bottleneck (compressed data) con una función no lineal
     keras.layers.Dense(13)
    ]
)

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Entrenar el modelo
historia = model.fit(x=df2_train,       # datos de entrenamiento
                     y=df2_train,       # etiquetas de entrenamiento
                     batch_size=32,     # tamaño del batch
                     epochs=50,         # numero de epocas
                     callbacks=lista_callbacks,             # callbacks
                     validation_data=(df2_val, df2_val))    # datos de validacion

In [ ]:
# Cargar el mejor modelo
model = keras.models.load_model("best_model.keras")

In [ ]:
# Acceder a la primera capa oculta (Bottleneck)

encoder = keras.Model(inputs=model.input, 
                      outputs=model.layers[0].output)

In [ ]:
df2_train_compressed = encoder.predict(df2_train) # comprimir los datos de entrenamiento
df2_val_compressed = encoder.predict(df2_val) # comprimir los datos de entrenamiento

df2_train_compressed.shape

In [ ]:
for clase in range(3):
    aux = (y_train == clase) # qué vinos son de la clase?
    plt.plot(df2_train_compressed[aux,0], 
             df2_train_compressed[aux,1], 'o', 
             label=clase)

Ahora se pueden ver las tres clases en colores diferentes.

## Calcular errores

In [ ]:
error_reconst = ((model.predict(df2_val)-df2_val.values)**2).sum(axis=1) # error de reconstruccion: suma de los errores cuadráticos de cada variable
error_reconst.shape

In [ ]:
indice_error_maximo = np.argmax(error_reconst) # índice del error máximo: el vino que peor se reconstruye
indice_error_maximo